In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
sys.path.append('..')
from fastai.vision import *
from audio import *
import matplotlib.pyplot as plt
import math

In [ ]:
data_url = 'http://www.openslr.org/resources/45/ST-AEDS-20180100_1-OS'
data_folder = datapath4file(url2name(data_url))
audios = AudioList.from_folder(data_folder)

config = AudioConfig()
config

config.duration = 4000

@torch.jit.script
@torch.no_grad()
def spec_augment(sgs:Tensor, size:int=20):
    bsg = sgs
    device = sgs.device
    bs, _, x, y = bsg.shape
    max_y = y-size-1
    rsize = torch.tensor(x*bs)
    m = torch.arange(y,device=device).repeat(rsize).view(bs, -1)
    rs = torch.randint(0, max_y,(bs,1), device=device)
    gpumask = ((m < rs) | (m > (rs+size))).view(bs, 1, x, -1)
    return bsg*gpumask

def newiter(self):
    for b in self.dl:
        b[0] = spec_augment(b[0])
        b[0] = spec_augment(b[0].transpose(2,3)).transpose(2,3)
        yield self.proc_batch(b)

DeviceDataLoader.__iter__ = newiter

In [ ]:
label_pattern = r'_([mf]\d+)_'
audios = AudioList.from_folder(data_folder, config=config).split_by_rand_pct(.2, seed=4).label_from_re(label_pattern)

tfms = get_spectro_transforms(mask_time=False, mask_frequency=False, roll=False, num_rows=5)
db = audios.transform(tfms).databunch(bs=64)
item = audios.train[0][0].spectro[None]
item = spec_augment(item)
item = spec_augment(item.transpose(2,3)).transpose(2,3)
plt.imshow(item[0][0])

learn = audio_learner(db)
learn.lr_find()
learn.recorder.plot()

learn.fit_one_cycle(5, slice(2e-3, 2e-2))

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()